In [36]:
import wfdb
import numpy as np
import pywt
from scipy.signal import find_peaks
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Input, Conv1D, Conv1DTranspose, LeakyReLU, PReLU, Add, BatchNormalization, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
import os


def load_mit_bih_data(record_path, load_annotation=True):
    record = wfdb.rdrecord(record_path)
    if load_annotation:
        annotation = wfdb.rdann(record_path, 'atr')
        return record.p_signal[:, 0], annotation
    else:
        return record.p_signal[:, 0], None


def add_noise(ecg_signal, noise_signal, snr):
    signal_power = np.mean(ecg_signal ** 2)
    noise_power = np.mean(noise_signal ** 2)
    scale_factor = np.sqrt(signal_power / (noise_power * 10 ** (snr / 10.0)))
    noisy_signal = ecg_signal + scale_factor * noise_signal
    return noisy_signal

# Pan-Tompkins Algorithm for QRS Detection
def pan_tompkins_qrs(ecg_signal, fs=360):
    diff = np.diff(ecg_signal)
    squared = diff ** 2
    integrated = np.convolve(squared, np.ones(5))
    peaks, _ = find_peaks(integrated, distance=fs*0.6)
    return peaks

# Segment Heartbeats based on QRS detection
def segment_heartbeats(ecg_signal, qrs_peaks, window_size=128):
    heartbeats = []
    for peak in qrs_peaks:
        # Extract heartbeat segment around the QRS peak
        start = max(peak - window_size // 2, 0)
        end = min(peak + window_size // 2, len(ecg_signal))
        heartbeat = ecg_signal[start:end]
        
        # If the segment is smaller than the window, pad it
        if len(heartbeat) < window_size:
            heartbeat = np.pad(heartbeat, (0, window_size - len(heartbeat)), 'constant')
        
        heartbeats.append(heartbeat)
    return np.array(heartbeats)


from sklearn.decomposition import PCA

# Adjust the wavelet decomposition function
def wavelet_decomposition(signal, wavelet='db4', level=4, target_length=512):
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    approx_coeffs = coeffs[0]  # Get the approximation coefficients
    
    current_length = len(approx_coeffs)
    
    if current_length > target_length:
        approx_coeffs = approx_coeffs[:target_length]
    elif current_length < target_length:
        approx_coeffs = np.pad(approx_coeffs, (0, target_length - current_length), 'constant')
    
    return approx_coeffs.reshape(target_length, 1)



# Build Generator Model
def build_generator(input_shape):
    input_layer = Input(shape=input_shape)  # Correct variable name

    # Encoder part
    x = Conv1D(64, kernel_size=15, strides=1, padding='same')(input_layer)  # Use input_layer instead of inputs
    x = PReLU()(x)
    x1 = Conv1D(128, kernel_size=15, strides=2, padding='same')(x)
    x1 = PReLU()(x1)
    x2 = Conv1D(256, kernel_size=15, strides=2, padding='same')(x1)
    x2 = PReLU()(x2)
    x3 = Conv1D(512, kernel_size=15, strides=2, padding='same')(x2)
    x3 = PReLU()(x3)
    x4 = Conv1D(512, kernel_size=15, strides=2, padding='same')(x3)
    x4 = PReLU()(x4)
    x5 = Conv1D(256, kernel_size=15, strides=2, padding='same')(x4)
    x5 = PReLU()(x5)
    x6 = Conv1D(128, kernel_size=15, strides=2, padding='same')(x5)
    x6 = PReLU()(x6)
    x7 = Conv1D(64, kernel_size=15, strides=2, padding='same')(x6)
    x7 = PReLU()(x7)
    
    # Decoder part with skip connections and 1x1 convolutions
    x7 = Conv1DTranspose(64, kernel_size=15, strides=2, padding='same')(x7)
    x6_resized = Conv1D(64, kernel_size=1, strides=1, padding='same')(x6)  # 1x1 conv to match dimensions
    x7 = Add()([x7, x6_resized])  # Skip connection
    x7 = PReLU()(x7)

    x7 = Conv1DTranspose(128, kernel_size=15, strides=2, padding='same')(x7)
    x5_resized = Conv1D(128, kernel_size=1, strides=1, padding='same')(x5)  # 1x1 conv to match dimensions
    x7 = Add()([x7, x5_resized])  # Skip connection
    x7 = PReLU()(x7)

    x7 = Conv1DTranspose(256, kernel_size=15, strides=2, padding='same')(x7)
    x4_resized = Conv1D(256, kernel_size=1, strides=1, padding='same')(x4)  # 1x1 conv to match dimensions
    x7 = Add()([x7, x4_resized])  # Skip connection
    x7 = PReLU()(x7)

    x7 = Conv1DTranspose(512, kernel_size=15, strides=2, padding='same')(x7)
    x3_resized = Conv1D(512, kernel_size=1, strides=1, padding='same')(x3)  # 1x1 conv to match dimensions
    x7 = Add()([x7, x3_resized])  # Skip connection
    x7 = PReLU()(x7)

    x7 = Conv1DTranspose(256, kernel_size=15, strides=2, padding='same')(x7)
    x2_resized = Conv1D(256, kernel_size=1, strides=1, padding='same')(x2)  # 1x1 conv to match dimensions
    x7 = Add()([x7, x2_resized])  # Skip connection
    x7 = PReLU()(x7)

    x7 = Conv1DTranspose(128, kernel_size=15, strides=2, padding='same')(x7)
    x1_resized = Conv1D(128, kernel_size=1, strides=1, padding='same')(x1)  # 1x1 conv to match dimensions
    x7 = Add()([x7, x1_resized])  # Skip connection
    x7 = PReLU()(x7)

    x7 = Conv1DTranspose(64, kernel_size=15, strides=2, padding='same')(x7)
    x_resized = Conv1D(64, kernel_size=1, strides=1, padding='same')(x)  # 1x1 conv to match dimensions
    x7 = Add()([x7, x_resized])  # Skip connection
    x7 = PReLU()(x7)

    outputs = Conv1DTranspose(1, kernel_size=15, strides=1, padding='same', activation='tanh')(x7)

    return Model(input_layer, outputs)  # Use input_layer as the input to the model

# Build Discriminator Model
def build_discriminator(input_shape=(512, 1)):
    input_layer = Input(shape=input_shape)
    
    # Convolutional layers
    x = Conv1D(64, kernel_size=3, strides=2, padding='same')(input_layer)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)

    x = Conv1D(128, kernel_size=3, strides=2, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)

    x = Conv1D(256, kernel_size=3, strides=2, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)

    x = Conv1D(512, kernel_size=3, strides=2, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)

    # Flatten and output
    x = Flatten()(x)
    output_layer = Dense(1, activation='sigmoid')(x)  # Output layer for binary classification
    
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# Compile the GAN
def compile_gan(generator, discriminator):
    # Use 'learning_rate' instead of 'lr'
    discriminator.compile(optimizer=RMSprop(learning_rate=0.0002), loss='binary_crossentropy', metrics=['accuracy'])
    discriminator.trainable = False

    gan_input = Input(shape=(generator.input_shape[1], generator.input_shape[2]))
    x = generator(gan_input)
    gan_output = discriminator(x)

    gan = Model(gan_input, gan_output)
    gan.compile(optimizer=RMSprop(learning_rate=0.0001), loss='binary_crossentropy')

    return gan

input_shape = (512, 1)

generator = build_generator(input_shape)
discriminator = build_discriminator(input_shape)
gan = compile_gan(generator, discriminator)

generator.summary()
discriminator.summary()

# Function to denoise the signals using the trained generator model
def denoise_signals(generator_model, signals):
    denoised_signals = generator_model.predict(signals)
    return denoised_signals

# Classification with SVM
def classify_heartbeats(features, labels):
    # Flatten the features from shape (n_samples, 512, 1) to (n_samples, 512)
    features_flattened = features.reshape(features.shape[0], features.shape[1])

    X_train, X_test, y_train, y_test = train_test_split(features_flattened, labels, test_size=0.2, random_state=42)
    svm_model = SVC(kernel='rbf')
    svm_model.fit(X_train, y_train)
    y_pred = svm_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy, y_pred


# Main Process
# Updated noise record path using 'bw' noise record
noise_record_path = 'C:\\Users\\malik\\Desktop\\Disertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\bw'

# Ensure the file exists
if not os.path.exists(f"{noise_record_path}.hea"):
    raise FileNotFoundError(f"Could not find file: {noise_record_path}.hea")

# Main function
def main():
    mit_bih_records = ['103', '105', '115']  # Add more records as needed

    all_heartbeats = []
    all_labels = []

    for record in mit_bih_records:
        # Load the ECG signal and annotation for each record
        ecg_signal, annotation = load_mit_bih_data(f'C:\\Users\\malik\\Desktop\\Disertation\\New folder\\mit-bih-arrhythmia-database-1.0.0/{record}')
        
        # Load noise signal
        noise_signal, _ = load_mit_bih_data(noise_record_path, load_annotation=False)

        # Add noise to the ECG signal
        noisy_signal = add_noise(ecg_signal, noise_signal[:len(ecg_signal)], snr=0)
        
        # Detect QRS peaks in the noisy signal
        qrs_peaks = pan_tompkins_qrs(noisy_signal)
        
        # Segment heartbeats based on the detected QRS peaks
        heartbeats = segment_heartbeats(noisy_signal, qrs_peaks)

        # Apply wavelet decomposition to each segmented heartbeat
        features = np.array([wavelet_decomposition(beat) for beat in heartbeats])

        # Generate labels for each heartbeat based on the QRS peaks and annotations
        labels = []
        for peak in qrs_peaks:
            idx = np.searchsorted(annotation.sample, peak)
            labels.append(annotation.symbol[idx] if idx < len(annotation.symbol) else 'N')

        # Append the extracted features and labels to the main lists
        all_heartbeats.extend(features)
        all_labels.extend(labels)

    # Convert the lists to numpy arrays
    all_heartbeats = np.array(all_heartbeats)
    all_labels = np.array(all_labels)

    # Ensure the shape of all_heartbeats is as expected
    print("Shape of all_heartbeats:", all_heartbeats.shape)

    # Define the number of epochs and batch size for GAN training
    epochs = 10
    batch_size = 64

    # GAN training loop
    for epoch in range(epochs):
        # Select a random batch of heartbeats
        idx = np.random.randint(0, all_heartbeats.shape[0], batch_size)
        real_heartbeats = all_heartbeats[idx].reshape(batch_size, 512, 1)  # Adjust shape as per the actual size

        # Optionally, add noise to the real heartbeats for training the generator
        noisy_real_heartbeats = add_noise(real_heartbeats, noise_signal[:len(real_heartbeats)], snr=0)

        # Training steps for the GAN would go here

    # Save the trained generator model
    generator.save('cae_cgan_generator.h5')

    # Use the generator model to denoise the heartbeats
    denoised_heartbeats = denoise_signals(generator, all_heartbeats)

    # Classify the denoised heartbeats using an SVM model
    accuracy, predictions = classify_heartbeats(denoised_heartbeats, all_labels)

    # Print the classification accuracy
    print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Ensure this line is present to run the main function
if __name__ == "__main__":
    main()


C:\Users\malik\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional_72"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_72 (InputLayer)   │ (None, 512, 1)            │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_456 (Conv1D)           │ (None, 512, 64)           │           1,024 │ input_layer_72[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ p_re_lu_360 (PReLU)           │ (None, 512, 64)           │          32,768 │ conv1d_456[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_457 (Conv1D)           │ (None, 256, 128)          │         123,008 │ p_re_lu_360[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ p_re_lu_361 (PReLU)           │ (None, 256, 128)          │          32,768 │ conv1d_457[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_458 (Conv1D)           │ (None, 128, 256)          │         491,776 │ p_re_lu_361[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ p_re_lu_362 (PReLU)           │ (None, 128, 256)          │          32,768 │ conv1d_458[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_459 (Conv1D)           │ (None, 64, 512)           │       1,966,592 │ p_re_lu_362[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ p_re_lu_363 (PReLU)           │ (None, 64, 512)           │          32,768 │ conv1d_459[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_460 (Conv1D)           │ (None, 32, 512)           │       3,932,672 │ p_re_lu_363[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ p_re_lu_364 (PReLU)           │ (None, 32, 512)           │          16,384 │ conv1d_460[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_461 (Conv1D)           │ (None, 16, 256)           │       1,966,336 │ p_re_lu_364[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ p_re_lu_365 (PReLU)           │ (None, 16, 256)           │           4,096 │ conv1d_461[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_462 (Conv1D)           │ (None, 8, 128)            │         491,648 │ p_re_lu_365[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ p_re_lu_366 (PReLU)           │ (None, 8, 128)            │           1,024 │ conv1d_462[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_463 (Conv1D)           │ (None, 4, 64)             │         122,944 │ p_re_lu_366[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ p_re_lu_367 (PReLU)           │ (None, 4, 64)             │             256 │ conv1d_463[0][0]           │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 15,137,025 (57.74 MB)

 Trainable params: 15,137,025 (57.74 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_73 (InputLayer)          │ (None, 512, 1)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_471 (Conv1D)                  │ (None, 256, 64)             │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_96 (LeakyReLU)           │ (None, 256, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_96               │ (None, 256, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_472 (Conv1D)                  │ (None, 128, 128)            │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_97 (LeakyReLU)           │ (None, 128, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_97               │ (None, 128, 128)            │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_473 (Conv1D)                  │ (None, 64, 256)             │          98,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_98 (LeakyReLU)           │ (None, 64, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_98               │ (None, 64, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_474 (Conv1D)                  │ (None, 32, 512)             │         393,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_99 (LeakyReLU)           │ (None, 32, 512)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_99               │ (None, 32, 512)             │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_24 (Flatten)                 │ (None, 16384)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (None, 1)                   │          16,385 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 537,473 (2.05 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 537,473 (2.05 MB)

Shape of all_heartbeats: (6588, 512, 1)
206/206 ━━━━━━━━━━━━━━━━━━━━ 131s 629ms/step
Classification Accuracy: 98.86%


In [ ]:
import wfdb
import numpy as np
import pywt
from scipy.signal import find_peaks
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Input, Conv1D, Conv1DTranspose, LeakyReLU, PReLU, Add, BatchNormalization, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
import os


def load_mit_bih_data(record_path, load_annotation=True):
    record = wfdb.rdrecord(record_path)
    if load_annotation:
        annotation = wfdb.rdann(record_path, 'atr')
        return record.p_signal[:, 0], annotation
    else:
        return record.p_signal[:, 0], None


def add_noise(ecg_signal, noise_signal, snr):
    signal_power = np.mean(ecg_signal ** 2)
    noise_power = np.mean(noise_signal ** 2)
    scale_factor = np.sqrt(signal_power / (noise_power * 10 ** (snr / 10.0)))
    noisy_signal = ecg_signal + scale_factor * noise_signal
    return noisy_signal

# Pan-Tompkins Algorithm for QRS Detection
def pan_tompkins_qrs(ecg_signal, fs=360):
    diff = np.diff(ecg_signal)
    squared = diff ** 2
    integrated = np.convolve(squared, np.ones(5))
    peaks, _ = find_peaks(integrated, distance=fs*0.6)
    return peaks

# Segment Heartbeats based on QRS detection
def segment_heartbeats(ecg_signal, qrs_peaks, window_size=128):
    heartbeats = []
    for peak in qrs_peaks:
        # Extract heartbeat segment around the QRS peak
        start = max(peak - window_size // 2, 0)
        end = min(peak + window_size // 2, len(ecg_signal))
        heartbeat = ecg_signal[start:end]
        
        # If the segment is smaller than the window, pad it
        if len(heartbeat) < window_size:
            heartbeat = np.pad(heartbeat, (0, window_size - len(heartbeat)), 'constant')
        
        heartbeats.append(heartbeat)
    return np.array(heartbeats)


from sklearn.decomposition import PCA

# Adjust the wavelet decomposition function
def wavelet_decomposition(signal, wavelet='db4', level=4, target_length=512):
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    approx_coeffs = coeffs[0]  # Get the approximation coefficients
    
    current_length = len(approx_coeffs)
    
    if current_length > target_length:
        approx_coeffs = approx_coeffs[:target_length]
    elif current_length < target_length:
        approx_coeffs = np.pad(approx_coeffs, (0, target_length - current_length), 'constant')
    
    return approx_coeffs.reshape(target_length, 1)



# Build Generator Model
def build_generator(input_shape):
    input_layer = Input(shape=input_shape)  # Correct variable name

    # Encoder part
    x = Conv1D(64, kernel_size=15, strides=1, padding='same')(input_layer)  # Use input_layer instead of inputs
    x = PReLU()(x)
    x1 = Conv1D(128, kernel_size=15, strides=2, padding='same')(x)
    x1 = PReLU()(x1)
    x2 = Conv1D(256, kernel_size=15, strides=2, padding='same')(x1)
    x2 = PReLU()(x2)
    x3 = Conv1D(512, kernel_size=15, strides=2, padding='same')(x2)
    x3 = PReLU()(x3)
    x4 = Conv1D(512, kernel_size=15, strides=2, padding='same')(x3)
    x4 = PReLU()(x4)
    x5 = Conv1D(256, kernel_size=15, strides=2, padding='same')(x4)
    x5 = PReLU()(x5)
    x6 = Conv1D(128, kernel_size=15, strides=2, padding='same')(x5)
    x6 = PReLU()(x6)
    x7 = Conv1D(64, kernel_size=15, strides=2, padding='same')(x6)
    x7 = PReLU()(x7)
    
    # Decoder part with skip connections and 1x1 convolutions
    x7 = Conv1DTranspose(64, kernel_size=15, strides=2, padding='same')(x7)
    x6_resized = Conv1D(64, kernel_size=1, strides=1, padding='same')(x6)  # 1x1 conv to match dimensions
    x7 = Add()([x7, x6_resized])  # Skip connection
    x7 = PReLU()(x7)

    x7 = Conv1DTranspose(128, kernel_size=15, strides=2, padding='same')(x7)
    x5_resized = Conv1D(128, kernel_size=1, strides=1, padding='same')(x5)  # 1x1 conv to match dimensions
    x7 = Add()([x7, x5_resized])  # Skip connection
    x7 = PReLU()(x7)

    x7 = Conv1DTranspose(256, kernel_size=15, strides=2, padding='same')(x7)
    x4_resized = Conv1D(256, kernel_size=1, strides=1, padding='same')(x4)  # 1x1 conv to match dimensions
    x7 = Add()([x7, x4_resized])  # Skip connection
    x7 = PReLU()(x7)

    x7 = Conv1DTranspose(512, kernel_size=15, strides=2, padding='same')(x7)
    x3_resized = Conv1D(512, kernel_size=1, strides=1, padding='same')(x3)  # 1x1 conv to match dimensions
    x7 = Add()([x7, x3_resized])  # Skip connection
    x7 = PReLU()(x7)

    x7 = Conv1DTranspose(256, kernel_size=15, strides=2, padding='same')(x7)
    x2_resized = Conv1D(256, kernel_size=1, strides=1, padding='same')(x2)  # 1x1 conv to match dimensions
    x7 = Add()([x7, x2_resized])  # Skip connection
    x7 = PReLU()(x7)

    x7 = Conv1DTranspose(128, kernel_size=15, strides=2, padding='same')(x7)
    x1_resized = Conv1D(128, kernel_size=1, strides=1, padding='same')(x1)  # 1x1 conv to match dimensions
    x7 = Add()([x7, x1_resized])  # Skip connection
    x7 = PReLU()(x7)

    x7 = Conv1DTranspose(64, kernel_size=15, strides=2, padding='same')(x7)
    x_resized = Conv1D(64, kernel_size=1, strides=1, padding='same')(x)  # 1x1 conv to match dimensions
    x7 = Add()([x7, x_resized])  # Skip connection
    x7 = PReLU()(x7)

    outputs = Conv1DTranspose(1, kernel_size=15, strides=1, padding='same', activation='tanh')(x7)

    return Model(input_layer, outputs)  # Use input_layer as the input to the model

# Build Discriminator Model
def build_discriminator(input_shape=(512, 1)):
    input_layer = Input(shape=input_shape)
    
    # Convolutional layers
    x = Conv1D(64, kernel_size=3, strides=2, padding='same')(input_layer)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)

    x = Conv1D(128, kernel_size=3, strides=2, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)

    x = Conv1D(256, kernel_size=3, strides=2, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)

    x = Conv1D(512, kernel_size=3, strides=2, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)

    # Flatten and output
    x = Flatten()(x)
    output_layer = Dense(1, activation='sigmoid')(x)  # Output layer for binary classification
    
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# Compile the GAN
def compile_gan(generator, discriminator):
    # Use 'learning_rate' instead of 'lr'
    discriminator.compile(optimizer=RMSprop(learning_rate=0.000001), loss='binary_crossentropy', metrics=['accuracy'])
    discriminator.trainable = False

    gan_input = Input(shape=(generator.input_shape[1], generator.input_shape[2]))
    x = generator(gan_input)
    gan_output = discriminator(x)

    gan = Model(gan_input, gan_output)
    gan.compile(optimizer=RMSprop(learning_rate=0.000005), loss='binary_crossentropy')

    return gan

input_shape = (512, 1)

generator = build_generator(input_shape)
discriminator = build_discriminator(input_shape)
gan = compile_gan(generator, discriminator)

generator.summary()
discriminator.summary()

# Function to denoise the signals using the trained generator model
def denoise_signals(generator_model, signals):
    denoised_signals = generator_model.predict(signals)
    return denoised_signals

# Classification with SVM
def classify_heartbeats(features, labels):
    # Flatten the features from shape (n_samples, 512, 1) to (n_samples, 512)
    features_flattened = features.reshape(features.shape[0], features.shape[1])

    X_train, X_test, y_train, y_test = train_test_split(features_flattened, labels, test_size=0.2, random_state=180)
    svm_model = SVC(kernel='rbf')
    svm_model.fit(X_train, y_train)
    y_pred = svm_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy, y_pred


# Main Process
# Updated noise record path using 'bw' noise record
noise_record_path = 'C:\\Users\\malik\\Desktop\\Disertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\bw'

# Ensure the file exists
if not os.path.exists(f"{noise_record_path}.hea"):
    raise FileNotFoundError(f"Could not find file: {noise_record_path}.hea")

# Main function
def main():
    mit_bih_records = ['103', '105', '111', '116', '122', '205', '213', '219', '223', '230']  # Add more records as needed

    all_heartbeats = []
    all_labels = []

    for record in mit_bih_records:
        # Load the ECG signal and annotation for each record
        ecg_signal, annotation = load_mit_bih_data(f'C:\\Users\\malik\\Desktop\\Disertation\\New folder\\mit-bih-arrhythmia-database-1.0.0/{record}')
        
        # Load noise signal
        noise_signal, _ = load_mit_bih_data(noise_record_path, load_annotation=False)

        # Add noise to the ECG signal
        noisy_signal = add_noise(ecg_signal, noise_signal[:len(ecg_signal)], snr=0)
        
        # Detect QRS peaks in the noisy signal
        qrs_peaks = pan_tompkins_qrs(noisy_signal)
        
        # Segment heartbeats based on the detected QRS peaks
        heartbeats = segment_heartbeats(noisy_signal, qrs_peaks)

        # Apply wavelet decomposition to each segmented heartbeat
        features = np.array([wavelet_decomposition(beat) for beat in heartbeats])

        # Generate labels for each heartbeat based on the QRS peaks and annotations
        labels = []
        for peak in qrs_peaks:
            idx = np.searchsorted(annotation.sample, peak)
            labels.append(annotation.symbol[idx] if idx < len(annotation.symbol) else 'N')

        # Append the extracted features and labels to the main lists
        all_heartbeats.extend(features)
        all_labels.extend(labels)

    # Convert the lists to numpy arrays
    all_heartbeats = np.array(all_heartbeats)
    all_labels = np.array(all_labels)

    # Ensure the shape of all_heartbeats is as expected
    print("Shape of all_heartbeats:", all_heartbeats.shape)

    # Define the number of epochs and batch size for GAN training
    epochs = 10000
    batch_size = 32

    # GAN training loop
    for epoch in range(epochs):
        # Select a random batch of heartbeats
        idx = np.random.randint(0, all_heartbeats.shape[0], batch_size)
        real_heartbeats = all_heartbeats[idx].reshape(batch_size, 512, 1)  # Adjust shape as per the actual size

        # Optionally, add noise to the real heartbeats for training the generator
        noisy_real_heartbeats = add_noise(real_heartbeats, noise_signal[:len(real_heartbeats)], snr=0)

        # Training steps for the GAN would go here

    # Save the trained generator model
    generator.save('cae_cgan_generator.h5')

    # Use the generator model to denoise the heartbeats
    denoised_heartbeats = denoise_signals(generator, all_heartbeats)

    # Classify the denoised heartbeats using an SVM model
    accuracy, predictions = classify_heartbeats(denoised_heartbeats, all_labels)

    # Print the classification accuracy
    print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Ensure this line is present to run the main function
if __name__ == "__main__":
    main()


C:\Users\malik\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)    │ (None, 512, 1)            │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_38 (Conv1D)            │ (None, 512, 64)           │           1,024 │ input_layer_6[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ p_re_lu_30 (PReLU)            │ (None, 512, 64)           │          32,768 │ conv1d_38[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_39 (Conv1D)            │ (None, 256, 128)          │         123,008 │ p_re_lu_30[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ p_re_lu_31 (PReLU)            │ (None, 256, 128)          │          32,768 │ conv1d_39[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_40 (Conv1D)            │ (None, 128, 256)          │         491,776 │ p_re_lu_31[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ p_re_lu_32 (PReLU)            │ (None, 128, 256)          │          32,768 │ conv1d_40[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_41 (Conv1D)            │ (None, 64, 512)           │       1,966,592 │ p_re_lu_32[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ p_re_lu_33 (PReLU)            │ (None, 64, 512)           │          32,768 │ conv1d_41[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_42 (Conv1D)            │ (None, 32, 512)           │       3,932,672 │ p_re_lu_33[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ p_re_lu_34 (PReLU)            │ (None, 32, 512)           │          16,384 │ conv1d_42[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_43 (Conv1D)            │ (None, 16, 256)           │       1,966,336 │ p_re_lu_34[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ p_re_lu_35 (PReLU)            │ (None, 16, 256)           │           4,096 │ conv1d_43[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_44 (Conv1D)            │ (None, 8, 128)            │         491,648 │ p_re_lu_35[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ p_re_lu_36 (PReLU)            │ (None, 8, 128)            │           1,024 │ conv1d_44[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_45 (Conv1D)            │ (None, 4, 64)             │         122,944 │ p_re_lu_36[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ p_re_lu_37 (PReLU)            │ (None, 4, 64)             │             256 │ conv1d_45[0][0]            │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 15,137,025 (57.74 MB)

 Trainable params: 15,137,025 (57.74 MB)

...

 Total params: 537,473 (2.05 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 537,473 (2.05 MB)

Shape of all_heartbeats: (23183, 512, 1)


344/725 ━━━━━━━━━━━━━━━━━━━━ 6:47 1s/step